In [4]:
import numpy as np
import pandas as pd
import geohash
PRECISION = 7

In [10]:
path = "~/Downloads/coronavirus_data/"

In [3]:
geo = pd.read_csv(path + 'locations.csv')

In [4]:
geo.head()

,user_id,x,y,time
0,c9025a55-9bfa-407a-93b1-d0bf14ba788e,2.620290,48.794475,1585654685
1,c9025a55-9bfa-407a-93b1-d0bf14ba788e,2.626324,48.802537,1585654647
2,30ce498e-9de5-43e4-b3d4-630b23a26cf1,2.346508,48.833758,1586124936
3,30ce498e-9de5-43e4-b3d4-630b23a26cf1,2.346492,48.833754,1586124968
4,81e1b50c-bda0-469d-bca6-7c6974a2d8f2,2.445168,48.955271,1586125526


In [5]:
targets =  pd.read_csv(path + 'infusr.csv')

In [6]:
geo['geohash'] = geo.apply(lambda x: geohash.encode(x['x'], x['y'], PRECISION), axis =  1)

In [7]:
geo['rounded_time'] = geo.apply(lambda x: int(x['time']/30), axis =  1)

In [8]:
from math import radians

In [9]:
def distance(lon1, lat1, lons, lats):
    lon1, lat1 = map(radians, [lon1, lat1])
    lons = np.radians(lons)
    lats = np.radians(lats)
    dlon = lons - lon1 
    dlat = lats - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lats) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371000 
    return c * r

In [10]:
distances = distance(55.733970, 37.642443, [55.735093, 55.734112], [37.643866, 37.645307])

In [11]:
distances > 200

array([False,  True])

In [12]:
geo.columns

Index(['user_id', 'x', 'y', 'time', 'geohash', 'rounded_time'], dtype='object')

In [13]:
result_dataset = []
for key, rows in geo.groupby(['geohash','rounded_time']):
    ids = []
    lons = []
    lats = []
    is_finalised = False
    for i, row in rows.iterrows():
        ids.append(row['user_id'])
        lons.append(row['x'])
        lats.append(row['y'])
    
    ids = np.array(ids)
    lons = np.array(lons)
    lats = np.array(lats)
    for i, row in rows.iterrows():
        close_ids = distance(row['x'], row['y'], lons, lats) > 15
        for id_, close in zip(ids, close_ids):
            if close and id_ != row['user_id']:
                result_dataset.append(
                        {
                            'time': row['rounded_time'], 
                            'user1': id_,
                            'user2': row['user_id']
                        }) 

In [14]:
dataset = pd.DataFrame(result_dataset)

In [16]:
dataset.shape

(5222, 3)

In [15]:
dataset.to_csv('contacts_15meters_30sec.csv')

In [18]:
dataset.head()

,time,user1,user2
0,158669838,834ca4e5-a1ee-4418-bdd5-93a1b5855a78,68f6349f-80e7-41cf-b025-d9cf0dc8d544
1,158669838,68f6349f-80e7-41cf-b025-d9cf0dc8d544,834ca4e5-a1ee-4418-bdd5-93a1b5855a78
2,158669928,68f6349f-80e7-41cf-b025-d9cf0dc8d544,834ca4e5-a1ee-4418-bdd5-93a1b5855a78
3,158669928,834ca4e5-a1ee-4418-bdd5-93a1b5855a78,68f6349f-80e7-41cf-b025-d9cf0dc8d544
4,158669936,68f6349f-80e7-41cf-b025-d9cf0dc8d544,834ca4e5-a1ee-4418-bdd5-93a1b5855a78


In [5]:
contacts = pd.read_csv('contacts_15meters_30sec.csv')

In [6]:
contacts.groupby(['user1', 'user2']).count().sort_values('time')

Unnamed: 0  \
user1                                user2                                              
01f6b034-d48f-4753-a1f7-9f6f3517e518 fb03d776-327f-44e5-a69d-b25523fb01f9           1   
611e9476-8b23-4999-9f65-dd4346dc177f 0f4b6787-a0cd-4d76-b7d0-a38d1657f19d           1   
                                     3cf6bf57-c44c-406b-9a52-5cc0ef3a717f           1   
6337ac00-309f-4f1c-83c3-51fedba647f9 f5c7d7d4-12e4-44d0-a24c-2821b6b638d1           1   
63940e0a-5753-4c27-9d81-1a3166b629f7 ef3745c0-c0a7-4b6c-b81f-6b18987c089d           1   
...                                                                               ...   
4c6c0f22-e558-40bb-8c79-2f46b0b3b745 6f63bdac-27fd-41a4-a3c9-551780c939d9         126   
9cba94a8-f3c4-43bc-a019-21f23bf32e20 2f798c1a-1708-467b-a14d-761d31e3b10c         265   
2f798c1a-1708-467b-a14d-761d31e3b10c 9cba94a8-f3c4-43bc-a019-21f23bf32e20         265   
5638a9e2-9189-4446-9711-36b3f2621ec2 604e9431-7ac0-4d25-9f1b-300ffb30ac99         509   
604e9431-7ac0-4d25-9f1b-300ffb30ac99 5638a9e2-9189-4446-9711-36b3f2621ec2         509   

                                                                           time  
user1                                user2                                       
01f6b034-d48f-4753-a1f7-9f6f3517e518 fb03d776-327f-44e5-a69d-b25523fb01f9     1  
611e9476-8b23-4999-9f65-dd4346dc177f 0f4b6787-a0cd-4d76-b7d0-a38d1657f19d     1  
                                     3cf6bf57-c44c-406b-9a52-5cc0ef3a717f     1  
6337ac00-309f-4f1c-83c3-51fedba647f9 f5c7d7d4-12e4-44d0-a24c-2821b6b638d1     1  
63940e0a-5753-4c27-9d81-1a3166b629f7 ef3745c0-c0a7-4b6c-b81f-6b18987c089d     1  
...                                                                         ...  
4c6c0f22-e558-40bb-8c79-2f46b0b3b745 6f63bdac-27fd-41a4-a3c9-551780c939d9   126  
9cba94a8-f3c4-43bc-a019-21f23bf32e20 2f798c1a-1708-467b-a14d-761d31e3b10c   265  
2f798c1a-1708-467b-a14d-761d31e3b10c 9cba94a8-f3c4-43bc-a019-21f23bf32e20   265  
5638a9e2-9189-4446-9711-36b3f2621ec2 604e9431-7ac0-4d25-9f1b-300ffb30ac99   509  
604e9431-7ac0-4d25-9f1b-300ffb30ac99 5638a9e2-9189-4446-9711-36b3f2621ec2   509  

[344 rows x 2 columns]

In [7]:
targets.shape

NameError: name 'targets' is not defined

In [8]:
user_targets = set(targets['user_id'].values)
contact_targets = set(contacts['user1'].values)
len(contact_targets & user_targets)

NameError: name 'targets' is not defined

### Данные о контактах:
-  С 6 по 20 апреля
- Много записей дублируются
- Необходимо отбирать уникальные за какое-то время для подсчета

### Данные о заражениях
- С 6 по 23 апреля
- Всего 650 случаев
- Каждая запись уникальна(нет записаей одного человека несколько раз)

Данные плохо пересекаются: среди контактных всего 70 зараженных

### Сбор выборки для обучения:
- Нужно сделать сетку(пусть в день)
- Если в этот день есть запись о инфицировании - человек в этот момент инфицирован(он инфицирован и далее, но в MVP только в этот день)
- Нужно собрать все контакты за предыдущую неделю
- Так как у нас данные с 6го апреля, сетку делаем с 13 по 23

In [44]:
targets =  pd.read_csv(path + 'infusr.csv')

In [12]:
import datetime
import ciso8601
import time

In [13]:
def get_timestamp(date_string):
    ts = ciso8601.parse_datetime(date_string)
    return int(time.mktime(ts.timetuple()))
START_DATE = get_timestamp("2020-04-13T23:59:59.000000")
END_DATE = get_timestamp("2020-04-24T23:59:59.000000")
DAY_LENGTH = 86400
# уникальные юзеры
unique_users = pd.unique(contacts['user1'])
# Формируем сетку
time_grid = np.arange(START_DATE, END_DATE, DAY_LENGTH)

In [14]:
START_DATE = get_timestamp("2020-04-13T23:59:59.000000")
END_DATE = get_timestamp("2020-04-24T23:59:59.000000")
DAY_LENGTH = 86400

In [15]:
# уникальные юзеры
unique_users = pd.unique(contacts['user1'])
# Формируем сетку
time_grid = np.arange(START_DATE, END_DATE, DAY_LENGTH)

In [16]:
def add_to_learning_grid(contacts_dict, time, user1, user2):
    key = user1 + '/' + str(time)
    if key not in contacts_dict.keys():
        contacts_dict[key] = set()
    contacts_dict[key].add(user2)

In [40]:
contacts_dict = {}
for i, row in contacts.iterrows():
    time_hash = row['time']
    time = time_hash*30
    user1 = row['user1']
    user2 = row['user2']
    close_time_elems = time_grid[((time_grid < time + 7*DAY_LENGTH) & (time_grid > time))]
    for time_elem in close_time_elems:
        add_to_learning_grid(contacts_dict, time_elem, user1, user2)
        

In [41]:
learning_list = []
for time_step in time_grid:
    for user in unique_users:
        dict_elem = {'user_id': user, 'grid_time': time_step}
        key = user + '/' + str(time_step)
        if key in contacts_dict.keys():
            dict_elem['contacts_num'] = len(contacts_dict[key])
        else:
            dict_elem['contacts_num']  = 0
        learning_list.append(dict_elem)

In [42]:
learning_data = pd.DataFrame(learning_list)

In [45]:
learning_data = learning_data.join(targets.set_index('user_id'), on='user_id', how='left')

In [46]:
learning_data.shape

(2750, 4)

In [47]:
def calc_target(inf_time, grid_time):
    if inf_time == inf_time:
        if 0 <= grid_time - inf_time <= 5*DAY_LENGTH:
            return 1
        if 0 < inf_time - grid_time <= 3*DAY_LENGTH:
            return 1
    return 0

In [48]:
learning_data['target'] = learning_data.apply(
    lambda x: calc_target(x['time'], x['grid_time']),  axis=1
)

In [49]:
import collections

In [50]:
frequency

Counter({1: 0.19527272727272726,
         3: 0.944,
         2: 0.7927272727272727,
         5: 0.9854545454545455,
         4: 0.9818181818181818,
         0: 0.0,
         7: 0.9934545454545455,
         8: 0.9974545454545455,
         6: 0.9934545454545455})

In [51]:
frequency = collections.Counter(learning_data['contacts_num'])
frequency[0] = learning_data[learning_data['contacts_num'] == 0].shape[0]
count = 0
for i in range(9):
    step = frequency[i]
    frequency[i] = count
    count += step
for i in range(9):
    if count == 0:
        frequency[i] = 0
    else:
        frequency[i] /= count

In [52]:
learning_data['predictions'] = learning_data.apply(
    lambda x: frequency[x['contacts_num']],  axis=1
)

In [53]:
learning_data

,user_id,grid_time,contacts_num,time,target,predictions
0,68f6349f-80e7-41cf-b025-d9cf0dc8d544,1586811599,1,NaN,0,0.450545
1,834ca4e5-a1ee-4418-bdd5-93a1b5855a78,1586811599,1,1.586676e+09,1,0.450545
2,50e0214b-1a2d-472b-82a8-30af93388f42,1586811599,0,NaN,0,0.000000
3,9a3e6b20-e639-404b-aead-a0ed1a3b0a00,1586811599,0,NaN,0,0.000000
4,dddb261a-9aef-4b09-af19-a99be7da261e,1586811599,0,NaN,0,0.000000
...,...,...,...,...,...,...
2745,2a5dab83-ed1d-406d-b060-e763ff80b258,1587675599,0,NaN,0,0.000000
2746,2899c279-a001-4d33-b101-3288efda5d1b,1587675599,0,NaN,0,0.000000
2747,4de25ebe-a9d5-4734-bed4-338d2439ad46,1587675599,2,NaN,0,0.885455
2748,a52868e5-617e-4364-8322-f80c534ecc18,1587675599,1,NaN,0,0.450545


### Получили результаты с предсказаниями

In [54]:
targets = learning_data['target'].values
predictions = learning_data['predictions'].values

In [55]:
for treshold in np.arange(0, 1, 0.05):
    tp = ((targets == 1) & (predictions > treshold )).sum()
    fp = ((targets == 0) & (predictions > treshold)).sum()
    tn = ((targets == 0) & (predictions < treshold )).sum()
    fn = ((targets == 1) & (predictions < treshold )).sum()
    print('treshold:', treshold, 'precision:',  tp/(tp + fp), 'recall:',  tp/(tp + fn))

treshold: 0.0 precision: 0.1502316346790205 recall: 1.0
treshold: 0.05 precision: 0.1502316346790205 recall: 0.6394366197183099
treshold: 0.1 precision: 0.1502316346790205 recall: 0.6394366197183099
treshold: 0.15000000000000002 precision: 0.1502316346790205 recall: 0.6394366197183099
treshold: 0.2 precision: 0.1502316346790205 recall: 0.6394366197183099
treshold: 0.25 precision: 0.1502316346790205 recall: 0.6394366197183099
treshold: 0.30000000000000004 precision: 0.1502316346790205 recall: 0.6394366197183099
treshold: 0.35000000000000003 precision: 0.1502316346790205 recall: 0.6394366197183099
treshold: 0.4 precision: 0.1502316346790205 recall: 0.6394366197183099
treshold: 0.45 precision: 0.1502316346790205 recall: 0.6394366197183099
treshold: 0.5 precision: 0.19682539682539682 recall: 0.17464788732394365
treshold: 0.55 precision: 0.19682539682539682 recall: 0.17464788732394365
treshold: 0.6000000000000001 precision: 0.19682539682539682 recall: 0.17464788732394365
treshold: 0.65 prec

In [29]:
for treshold in np.arange(0, 1, 0.05):
    tp = ((targets == 1) & (predictions > treshold )).sum()
    fp = ((targets == 0) & (predictions > treshold)).sum()
    tn = ((targets == 0) & (predictions < treshold )).sum()
    fn = ((targets == 1) & (predictions < treshold )).sum()
    print('treshold:', treshold, 'precision:',  tp/(tp + fp), 'recall:',  tp/(tp + fn))

treshold: 0.0 precision: 0.157704473565296 recall: 1.0
treshold: 0.05 precision: 0.157704473565296 recall: 0.9830985915492958
treshold: 0.1 precision: 0.157704473565296 recall: 0.9830985915492958
treshold: 0.15000000000000002 precision: 0.157704473565296 recall: 0.9830985915492958
treshold: 0.2 precision: 0.18596491228070175 recall: 0.29859154929577464
treshold: 0.25 precision: 0.18596491228070175 recall: 0.29859154929577464
treshold: 0.30000000000000004 precision: 0.18596491228070175 recall: 0.29859154929577464
treshold: 0.35000000000000003 precision: 0.18596491228070175 recall: 0.29859154929577464
treshold: 0.4 precision: 0.18596491228070175 recall: 0.29859154929577464
treshold: 0.45 precision: 0.18596491228070175 recall: 0.29859154929577464
treshold: 0.5 precision: 0.18596491228070175 recall: 0.29859154929577464
treshold: 0.55 precision: 0.18596491228070175 recall: 0.29859154929577464
treshold: 0.6000000000000001 precision: 0.18596491228070175 recall: 0.29859154929577464
treshold: 0

In [56]:
learning_data_new = learning_data[
    ((learning_data['contacts_num'] > 0) | (learning_data['target'] == 1))]
targets = learning_data_new['target'].values
predictions = learning_data_new['predictions'].values

In [78]:
targets = pd.read_csv('coronavirus/y_test.csv')
features = pd.read_csv('coronavirus/x_test.csv')

In [79]:
features['predictions'] = features.apply(
    lambda x: frequency[x['contacts_num']],  axis=1
)

In [81]:
targets = targets['target'].values
predictions = features['predictions'].values

In [76]:
features['predictions'].values.shape

(825,)

In [82]:
for treshold in np.arange(0, 1, 0.05):
    tp = ((targets == 1) & (predictions > treshold )).sum()
    fp = ((targets == 0) & (predictions > treshold)).sum()
    tn = ((targets == 0) & (predictions < treshold )).sum()
    fn = ((targets == 1) & (predictions < treshold )).sum()
    print('treshold:', treshold, 'precision:',  tp/(tp + fp), 'recall:',  tp/(tp + fn))

treshold: 0.0 precision: 0.125 recall: 1.0
treshold: 0.05 precision: 0.125 recall: 0.6333333333333333
treshold: 0.1 precision: 0.125 recall: 0.6333333333333333
treshold: 0.15000000000000002 precision: 0.125 recall: 0.6333333333333333
treshold: 0.2 precision: 0.125 recall: 0.6333333333333333
treshold: 0.25 precision: 0.125 recall: 0.6333333333333333
treshold: 0.30000000000000004 precision: 0.125 recall: 0.6333333333333333
treshold: 0.35000000000000003 precision: 0.125 recall: 0.6333333333333333
treshold: 0.4 precision: 0.125 recall: 0.6333333333333333
treshold: 0.45 precision: 0.125 recall: 0.6333333333333333
treshold: 0.5 precision: 0.15789473684210525 recall: 0.16666666666666666
treshold: 0.55 precision: 0.15789473684210525 recall: 0.16666666666666666
treshold: 0.6000000000000001 precision: 0.15789473684210525 recall: 0.16666666666666666
treshold: 0.65 precision: 0.15789473684210525 recall: 0.16666666666666666
treshold: 0.7000000000000001 precision: 0.15789473684210525 recall: 0.16666

### Лучший порог 0.75
treshold: 0.75 precision: 0.18596491228070175 recall: 0.29859154929577464

In [343]:
treshold = 0.75
tp = ((targets == 1) & (predictions > treshold )).sum()
fp = ((targets == 0) & (predictions > treshold)).sum()
tn = ((targets == 0) & (predictions < treshold )).sum()
fn = ((targets == 1) & (predictions < treshold )).sum()
print(tp, fp, tn, fn)

106 464 1931 249
